<a href="https://colab.research.google.com/github/bmanikan/FallDetection/blob/main/Code%20notebooks/CMDFALL/FallDetection_CMDFALL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data

In [ ]:
import json

kaggle = {"username":"bmanikan","key":"141e02fe5c7642ce9e22bf6ebf29e911"}

with open('kaggle.json', 'w') as outfile:
  json.dump(kaggle,outfile)
 

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

     |████████████████████████████████| 58 kB 2.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73052 sha256=0b7009700ff20672e25143ff713383e41cb0ae27c2fe3dd59c3100cd09a593c4
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
kaggle.json


In [ ]:
import os
os.makedirs('/content/dataset')

In [ ]:
!kaggle datasets download bmanikan/cmdfall
!unzip /content/cmdfall.zip -d dataset/cmdfall
os.remove('cmdfall.zip')

!kaggle datasets download bmanikan/cmdfall1
!unzip /content/cmdfall1.zip -d dataset/cmdfall
os.remove('cmdfall1.zip')

!kaggle datasets download bmanikan/cmdfall2
!unzip /content/cmdfall2.zip -d dataset/cmdfall
os.remove('cmdfall2.zip')





## classification


In [ ]:
import urllib.request as urlreq
import os
import cv2
import numpy as np
from pathlib import Path
import torch
from torch import cuda
from tqdm.notebook import tqdm
from zipfile import ZipFile
import torchvision
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn as fasterrcnn
import time
import shutil
import tensorflow as tf
import numpy as np
import random
import os

In [ ]:

#Set seed value
seed_value = 43

os.environ['PYTHONHASHSEED'] = str(seed_value)

random.seed(seed_value)

#numpy seed
np.random.seed(seed_value)

#Tf seed
tf.random.set_seed(seed_value)

#Configure new global tensorflow session
from tensorflow.compat.v1.keras import backend as k
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads = 1,
    inter_op_parallelism_threads = 1
)

sess = tf.compat.v1.Session(graph = tf.compat.v1.get_default_graph(), config = session_conf)
k.set_session(sess)

In [ ]:
%%capture
!pip install wandb

In [ ]:
import wandb 
wandb.login()
run = wandb.init(project="fall_detection",
          config={
              'frameCount'    :50,
              'data_size'     :180,
              'dataset'       :'CMDFALL',
              'epochs'        :15,
              'seed_value'    :43,
              'split'         :0.8,
              'batch_size'    :32,
              'activation'    :'relu',
              'optimizer'     :'adam',
              'loss_fn'       :'binarycrossentropy',
              'dropout_rate'  :0.5,
              'train_split'   :0.7,
              'val_split'     :0.2,
              'test_split'    :0.1
          })
cfg = wandb.config

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
artifact = run.use_artifact('trajectory:latest', type='dataset')
artifact_dir = artifact.download()

In [ ]:
artifact = run.use_artifact('boundingBox:latest', type='dataset')
artifact_dir = artifact.download()

wandb: Downloading large artifact boundingBox:latest, 402.29MB. 8423 files... Done. 0:0:0


## Logger

In [ ]:
import datetime
import time
import logging
import sys

def set_logger():
  logging.basicConfig(level=logging.INFO)
  logger_t = logging.getLogger('withoutlevel')
  logger = logging.getLogger('withlevel')

  f_handler = logging.FileHandler('Falldetection.log')
  f_handler_t = logging.FileHandler('Falldetection.log')

  f_format = logging.Formatter('%(levelname)s - %(message)s')
  f_format_t = logging.Formatter('%(message)s')

  f_handler.setFormatter(f_format)
  f_handler_t.setFormatter(f_format_t)

  logger.addHandler(f_handler)
  logger_t.addHandler(f_handler_t)
  return logger, logger_t

In [ ]:
import datetime
import time
t_set = lambda: datetime.datetime.now().astimezone()
t_diff = lambda t: str(t_set() - t)
w_diff = lambda t: (t_set() - t).total_seconds()
t_stamp = lambda t=None: str(t) if t else str(t_set())
t = t_set()
time.sleep(5)
t_diff(t)

'0:00:05.006115'

In [ ]:
import logging
import sys

#logger without time
logging.basicConfig(level=logging.INFO)
logger_t = logging.getLogger('withoutlevel')
logger = logging.getLogger('withlevel')

f_handler = logging.FileHandler('Falldetection.log')
f_handler_t = logging.FileHandler('Falldetection.log')

f_format = logging.Formatter('%(levelname)s - %(message)s')
f_format_t = logging.Formatter('%(message)s')

f_handler.setFormatter(f_format)
f_handler_t.setFormatter(f_format_t)

logger.addHandler(f_handler)
logger_t.addHandler(f_handler_t)


In [ ]:
import platform
platform.uname()
for det in platform.uname():
  logger_t.info(str(det))

INFO:withoutlevel:Linux
INFO:withoutlevel:42e36c1475b6
INFO:withoutlevel:5.4.104+
INFO:withoutlevel:#1 SMP Sat Jun 5 09:50:34 PDT 2021
INFO:withoutlevel:x86_64
INFO:withoutlevel:x86_64


In [ ]:
logger_t.info('='*100)
logger_t.info("\nFalldetection project")

INFO:withoutlevel:====================================================================================================
INFO:withoutlevel:
Falldetection project


In [ ]:
logger.info(f"The analysis with dataset {cfg.dataset} and frame count {cfg.frameCount} and size of the data is {cfg.data_size}.")

INFO:withlevel:The analysis with dataset CMDFALL and frame count 50 and size of the data is 180.


## Prepare dataframe

In [ ]:
import pandas as pd
df_actions = pd.read_csv('http://mica.edu.vn:8000/KinectData/public/action_list.txt')
actions = dict(list(df_actions.to_records(index=False)))
df_frames = pd.read_csv('http://mica.edu.vn:8000/KinectData/public/annotation.csv')

df_video = df_frames[['setup_id','subject_id','kinect_id']]
df_video = df_video.drop_duplicates(keep='first',ignore_index=True)


In [ ]:
# Cleaning the dataframe
df_frames.loc[df_frames['action_id'] == '\ufeff1', 'action_id'] = 1
df_frames.loc[df_frames['start_frame'] == '89s59','start_frame'] = 8959
# Encoding the dataframe
df_frames['start_frame'] = df_frames['start_frame'].apply(pd.to_numeric)
df_frames['action_id'] = df_frames['action_id'].apply(pd.to_numeric)
# details of the action the human performing
df_frames['action_detail'] = df_frames['action_id'].apply(lambda x: actions[x])
# details of supercategory the action resides 
df_frames['action'] = df_frames['action_id'].apply(lambda x: 'fall' if 'fall' in actions[x] else 'adl')
# Encoding the action column
df_frames['action_label'] = df_frames['action_id'].apply(lambda x: 1 if 'fall' in actions[x] else 0)

In [ ]:

df_frames['frame_count'] = df_frames['stop_frame'] - df_frames['start_frame']

# Print some statistics
print(f"Total videos possible in this dataset is {len(df_frames)}")
print(f"Number of fall videos in the dataset is {len(df_frames[df_frames.action == 'fall'])}")
print(f"Number of adl videos in the dataset is {len(df_frames[df_frames.action == 'adl'])}")
print(f"Videos greater than 50 Frames are {len(df_frames[df_frames.frame_count > 50])}")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


Total videos possible in this dataset is 14029
Number of fall videos in the dataset is 5472
Number of adl videos in the dataset is 8557
Videos greater than 50 Frames are 9481


In [ ]:
df_frames.head()

,setup_id,subject_id,kinect_id,action_id,start_frame,stop_frame,action_detail,action,action_label,frame_count
0,1,19,2,1,0,216,walk,adl,0,216
1,1,19,2,1,286,449,walk,adl,0,163
2,1,19,2,1,584,957,walk,adl,0,373
3,1,19,2,2,960,1343,run_slowly,adl,0,383
4,1,19,2,3,1344,1746,static_jump,adl,0,402


In [ ]:
actions

{1: 'walk',
 2: 'run_slowly',
 3: 'static_jump',
 4: 'move_hand_and_leg',
 5: 'left_hand_pick_up',
 6: 'right_hand_pick_up',
 7: 'stagger',
 8: 'front_fall',
 9: 'back_fall',
 10: 'left_fall',
 11: 'right_fall',
 12: 'crawl',
 13: 'sit_on_chair_then_stand_up',
 14: 'move_chair',
 15: 'sit_on_chair_then_fall_left',
 16: 'sit_on_chair_then_fall_right',
 17: 'sit_on_bed_and_stand_up',
 18: 'lie_on_bed_and_sit_up',
 19: 'lie_on_bed_and_fall_left',
 20: 'lie_on_bed_and_fall_right'}

## Download entire dataset

In [ ]:
def safedirs(path):
  if not os.path.exists(path):
    os.makedirs(path)

root = os.getcwd()
cmdfall = os.path.join(root,'dataset','cmdfall')


In [ ]:
values = list(df_video.iloc[0])
url = f'http://mica.edu.vn:8000/KinectData/public/colors/S{values[0]}P{values[1]}K{values[2]}.avi'
urlreq.urlretrieve(url, 'video.avi')
subframes = df_frames[(df_frames.setup_id == values[0])
                     & (df_frames.subject_id == values[1])
                     & (df_frames.kinect_id == values[2])]



In [ ]:
import time

t = time.time()
for video_idx in range(len(df_video)):
  values = list(df_video.iloc[video_idx])
  video_filename = f'S{values[0]}P{values[1]}K{values[2]}' 
  url = f'http://mica.edu.vn:8000/KinectData/public/colors/S{values[0]}P{values[1]}K{values[2]}.avi'
  video_temp_path = os.path.join(cmdfall,'temp')
  safedirs(video_temp_path)
  urlreq.urlretrieve(url, os.path.join(video_temp_path,f'{video_filename}.avi'))
  subframes = df_frames[(df_frames.setup_id == values[0])
                     & (df_frames.subject_id == values[1])
                     & (df_frames.kinect_id == values[2])]
  counter = 0
  cap = cv2.VideoCapture(os.path.join(video_temp_path, f'{video_filename}.avi'))
  frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
  print(frame_count)
  while (cap.isOpened()):
    ret,frame = cap.read()
    if ret == False:
      break
    frame_details = subframes[(subframes.start_frame <= counter) & (subframes.stop_frame >= counter)].values.tolist()
    if not frame_details:
      #print(f'skipping.... {counter}')
      counter += 1
      continue
    action_detail, action, action_label = frame_details[0][-3:]
    action_dest_path = os.path.join(cmdfall, 'frame', video_filename, action, action_detail)
    safedirs(action_dest_path)
    file_path = os.path.join(action_dest_path, f'{video_filename}_{counter}_{action_label}.png')
    cv2.imwrite(file_path, frame)
    counter += 1
    if counter >= frame_count:
      cap.release()
      break
  break

print(time.time() - t)

10270.0
161.11956071853638


In [ ]:
for temp in os.listdir('/content/dataset/cmdfall/frame/S1P19K2/adl'):
  print(temp, len(os.listdir('/content/dataset/cmdfall/frame/S1P19K2/adl/'+temp)))

move_chair 487
sit_on_bed_and_stand_up 286
left_hand_pick_up 233
static_jump 403
sit_on_chair_then_stand_up 224
move_hand_and_leg 391
stagger 391
walk 755
lie_on_bed_and_sit_up 152
right_hand_pick_up 218
crawl 155
run_slowly 384


## Download videos

In [ ]:
# video sequence that has greater than specified frame count is taken into account
df_frames = df_frames[df_frames.frame_count > cfg['frameCount']]
# Shuffle and retrieve the subset of Fall and Adl samples
df_fall = df_frames[df_frames.action == 'fall'].sample(frac=1,random_state=cfg['seed_value']).reset_index(drop=True)[:cfg['data_size']//2]
df_adl = df_frames[df_frames.action == 'adl'].sample(frac=1,random_state=cfg['seed_value']).reset_index(drop=True)[:cfg['data_size']//2]
df_sample = pd.concat([df_fall,df_adl]).sample(frac=1, random_state=cfg['seed_value']).reset_index(drop=True)


In [ ]:
df_fall.action_detail.unique()

array(['left_fall', 'back_fall', 'front_fall', 'right_fall',
       'lie_on_bed_and_fall_left', 'lie_on_bed_and_fall_right',
       'sit_on_chair_then_fall_right', 'sit_on_chair_then_fall_left'],
      dtype=object)

In [ ]:
df_sample.head()

,setup_id,subject_id,kinect_id,action_id,start_frame,stop_frame,action_detail,action,action_label,frame_count
0,38,16,4,14,6412,6782,move_chair,adl,0,370
1,30,7,6,12,5509,5642,crawl,adl,0,133
2,18,6,2,19,9904,9995,lie_on_bed_and_fall_left,fall,1,91
3,3,30,3,15,6735,6790,sit_on_chair_then_fall_left,fall,1,55
4,25,39,5,5,2627,2686,left_hand_pick_up,adl,0,59


In [ ]:
def safedirs(path):
  if not os.path.exists(path):
    os.makedirs(path)

logger_t.info(f'{"="*100} \n\nSTEP 1: Download dataset into the system')

root = os.getcwd()
# CMDFALL dataset video path
cmdfall = os.path.join(root,'dataset','cmdfall')
# path for subset of frames stored
subset = os.path.join(root, 'subset')

logger.info(f" dataset is downloaded in path {cmdfall}")



INFO:withoutlevel:==================================================================================================== 

STEP 1: Download dataset into the system
INFO:withlevel: dataset is downloaded in path /content/dataset/cmdfall


In [ ]:
shutil.rmtree('/content/subset')

In [ ]:
def download_dataset(df_sample, cmdfall, subset):
  # data size
  train_size = int(cfg.train_split * len(df_sample))
  validation_size = int(cfg.val_split * len(df_sample)) + train_size
  test_size = int(cfg.test_split * len(df_sample)) + validation_size
  logger.info(f'Number of files in Train set is {train_size}')
  logger.info(f'Number of files in validation set is {validation_size - train_size}')
  logger.info(f'Number of files in Test set is {test_size - validation_size}')
  # each video sequence from the data frame
  for video_idx in tqdm(range(len(df_sample))):
    t = t_set()
    # downloaded to train, validation and test in sequence
    split = 'train' if video_idx in range(train_size) else 'val' if video_idx in range(validation_size) else 'test'
    # for each record in datframe
    values = df_sample.iloc[video_idx]
    video_filename = f'S{values.setup_id}P{values.subject_id}K{values.kinect_id}' 
    url = f'http://mica.edu.vn:8000/KinectData/public/colors/{video_filename}.avi'
    # folder to store videos
    video_temp_folder = os.path.join(cmdfall,'temp')
    video_temp_file = os.path.join(video_temp_folder,f'{video_filename}.avi')
    # download video if it is not already downloaded
    if not os.path.exists(video_temp_file):
      safedirs(video_temp_folder)
      urlreq.urlretrieve(url, video_temp_file)
    # folder to store the frames of the video
    frame_folder = os.path.join(subset, split, values.action, video_filename)
    # if the frame set is already available with that video it appends the name with counter
    counter = 1
    while os.path.exists(frame_folder):
      frame_folder = os.path.join(subset,split, values.action, video_filename + f'_{counter}')
      counter += 1
    safedirs(frame_folder)
    logger_t.info(f'The file is downloaded from {url} \n extracted into path ...{frame_folder}')
    # store path of video in dataframe
    df_frames.loc[(df_frames.setup_id == values.setup_id)
                & (df_frames.subject_id == values.subject_id)
                & (df_frames.kinect_id == values.kinect_id), 'video_path'] = video_temp_file
    # read the video file
    cap = cv2.VideoCapture(video_temp_file)
    total_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # randomly select the 50 frames from the video sequence
    random_start = random.randint(values.start_frame, values.stop_frame - cfg['frameCount'])
    random_sample = sorted(random.sample(range(values.start_frame, values.stop_frame+1), cfg['frameCount']))
    # read specific frames from the video without reading it sequence
    print(values.start_frame, values.stop_frame)
    for frame_idx in range(values.start_frame, values.stop_frame+1):
      string_idx = str(frame_idx).zfill(6)
      frame_file_path = os.path.join(frame_folder, f'{video_filename}_{string_idx}_{values.action_label}.jpg')
      cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
      ret,frame = cap.read()
      if not ret:
        break
        raise Exception
      #print(frame_file_path)
      cv2.imwrite(frame_file_path, frame)
    logger.info(f'Elapsed time for downloading and extracting the file is {t_diff(t)}.\n')
    wandb.log({'download':{'time': w_diff(t)}})

t = t_set()
download_dataset(df_sample, cmdfall, subset)
logger.info(f'Total time for downloading and extracting entire dataset is {t_diff(t)}\n {"="*200}')
wandb.summary['Download_TotalTime'] = w_diff(t)

INFO:withlevel:Number of files in Train set is 125
INFO:withlevel:Number of files in validation set is 36
INFO:withlevel:Number of files in Test set is 18


INFO:withoutlevel:The file is downloaded from http://mica.edu.vn:8000/KinectData/public/colors/S38P16K4.avi 
 extracted into path .../content/subset/train/adl/S38P16K4


6412 6782


INFO:withlevel:Elapsed time for downloading and extracting the file is 0:00:10.865405.

INFO:withoutlevel:The file is downloaded from http://mica.edu.vn:8000/KinectData/public/colors/S30P7K6.avi 
 extracted into path .../content/subset/train/adl/S30P7K6


5509 5642


...

INFO:withlevel:Elapsed time for downloading and extracting the file is 0:00:02.972138.

INFO:withoutlevel:The file is downloaded from http://mica.edu.vn:8000/KinectData/public/colors/S18P6K1.avi 
 extracted into path .../content/subset/test/fall/S18P6K1


9562 9648


INFO:withlevel:Elapsed time for downloading and extracting the file is 0:00:02.603117.

INFO:withoutlevel:The file is downloaded from http://mica.edu.vn:8000/KinectData/public/colors/S17P21K5.avi 
 extracted into path .../content/subset/test/fall/S17P21K5


5088 5169


INFO:withlevel:Elapsed time for downloading and extracting the file is 0:00:02.343620.

INFO:withlevel:Total time for downloading and extracting entire dataset is 0:44:38.689273


In [ ]:
starting_frame

{'S13P42K3': '/content/subset/test/fall/S13P42K3/S13P42K3_006907_1.jpg',
 'S16P41K4': '/content/subset/test/fall/S16P41K4/S16P41K4_009503_1.jpg',
 'S17P21K5': '/content/subset/test/fall/S17P21K5/S17P21K5_005088_1.jpg'}

In [ ]:
import glob
fall_frames_list = sorted(glob.glob('/content/subset/*/fall/*'))
starting_frame = dict()
for L in fall_frames_list:
  starting_frame[os.path.basename(L)] = input(f'Starting frame for {L} ')

## Embedding data

In [ ]:
from collections import defaultdict
from skimage.metrics import structural_similarity
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time,os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing


def person_detector(imageMatrix,label,mode,folder):
  '''
  Moving object detection by frame differencing
  input: torch tensor object of shape [batch,50,3,480,640] and labels array
  output: tuple of torch tensor [batch,1,224,224] and label object 
  '''
  info = ['adl','fall']
  bbox_path = os.path.join(os.getcwd(), 'boundingBox', mode, info[label], folder)
  logger_t.info(f'Working on creating ... {bbox_path} set')
  safedirs(bbox_path)
  temp_img = imageMatrix
  center_points = []
  # use structure similarity for detecting the motion between the frames
  for i in range(0,temp_img.shape[0] - 2):
    img1 = cv2.medianBlur(cv2.cvtColor(temp_img[i],cv2.COLOR_BGR2GRAY),5)
    img2 = cv2.medianBlur(cv2.cvtColor(temp_img[i+2],cv2.COLOR_BGR2GRAY),5)
    # score and differences in the pixel
    (score,diff) = structural_similarity(img1,img2,full=True)
    diff = (diff * 255).astype('uint8')
    # dilate the contours to avoid noise
    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    dilate_frame = cv2.dilate(thresh, None, iterations=5)
    contours = cv2.findContours(dilate_frame.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1] 
    # Calculate the contour area
    contourareas = [cv2.contourArea(c) for c in contours]
    # skip if the there is not motion detected
    if not contourareas:
      print(f'skipping {bbox_path}')
      continue
    # get the contour which has maximum area
    val = np.argmax(contourareas)
    # draw the bounding box around the contour
    x,y,w,h = cv2.boundingRect(contours[val])
    # draw the bounding box in the image
    cv2.rectangle(img1, (x, y), (x + w, y + h), (36,255,12), 2)
    str_idx = str(i).zfill(3)
    path = os.path.join(bbox_path, f'image_{str_idx}.jpg')
    # store the intermediate file
    cv2.imwrite(path, img1)
    # calculate the centers of the boxes are return it
    center = (int((x+w)/2), int((y+h)/2))
    center_points.append(center)
  return center_points

def get_transform():
  transform = tf.keras.Sequential([
                                   preprocessing.Resizing(224,224),
                                   preprocessing.Normalization()
  ])
  return transform


def image_embedding(images,label,mode,filename):
  # transforms for resizing and normalizing image
  transform = get_transform()
  info = ['adl', 'fall']
  # path to store trajectory files 
  trajectory_path = os.path.join(os.getcwd(), 'trajectories', mode, info[label])
  safedirs(trajectory_path)
  logger_t.info(f'creating trajectory image at {trajectory_path}')
  bkg = np.zeros((480,640),np.uint8)
  t=t_set()
  # retrieve the center points of the bounding rectangle
  points = person_detector(images,label,mode,filename) 
  logger.info(f'Elapsed time for Bounding box calculation is {t_diff(t)}')
  wandb.log({'boundingBox':{'time':w_diff(t)}})
  # draw line based on the points to get trajectory
  for p in range(len(points)-1):
    bkg = cv2.line(bkg,points[p],points[p+1],(255,0,0),5)
  bkg = transform(tf.expand_dims(bkg,2))
  path = os.path.join(trajectory_path, filename+'.jpg')
  # store the files in the folder
  cv2.imwrite(path, bkg.numpy())


def embedded_data(datapath):
  data_dict = defaultdict(list)
  # walkthrough all the files in the datafolder and calculate bounding box
  for root,dir,files in os.walk(datapath):
      for f in files:
        frame_path = os.path.join(root,f)
        #print(root)
        folder = os.path.basename(root)
        data_dict[folder].append(frame_path)
  # encode each image in the folder
  for key in list(data_dict.keys()):
    frames, labels = [], []
    for f in sorted(data_dict[key]):
      labels.append(int(f.split('.')[0][-1]))
      temp = f.split('/')
      mode = temp[3]
      filename = temp[5]
      img = cv2.cvtColor(cv2.imread(f),cv2.COLOR_BGR2RGB)
      frames.append(img)
    frames = np.stack(frames,axis=0)
    t=t_set()
    image_embedding(frames, np.max(labels), mode, filename)
    logger.info(f"Total time taken for {key} is {t_diff(t)}")
    wandb.log({'trajectory':{'time':w_diff(t)}})


logger_t.info(f'{"="*100} \n\nSTEP 4: Create Trajectory images\n')
t = t_set()
root = os.getcwd()
embedded_data(root + '/subset/train')
embedded_data(root + '/subset/val')
embedded_data(root + '/subset/test')
logger.info(f'Total time taken to create trajectory images: {t_diff(t)}')
wandb.summary["Trajectory_totaltime"] = w_diff(t)


INFO:withoutlevel:==================================================================================================== 

STEP 4: Create Trajectory images

INFO:withoutlevel:creating trajectory image at /content/trajectories/train/fall
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/fall/S24P37K2 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.350906
INFO:withlevel:Total time taken for S24P37K2 is 0:00:02.385328
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/fall
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/fall/S44P5K4 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.388301
INFO:withlevel:Total time taken for S44P5K4 is 0:00:02.420856
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/fall
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/fall/S30P7K1 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.368

skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall

INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.288128
INFO:withlevel:Total time taken for S4P2K7 is 0:00:02.324871


skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7
skipping /content/boundingBox/train/fall/S4P2K7


INFO:withoutlevel:creating trajectory image at /content/trajectories/train/fall
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/fall/S35P11K2 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.358674
INFO:withlevel:Total time taken for S35P11K2 is 0:00:02.391244
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/fall
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/fall/S2P32K2 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.329495
INFO:withlevel:Total time taken for S2P32K2 is 0:00:02.361556
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/fall
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/fall/S18P6K5 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:04.839848
INFO:withlevel:Total time taken for S18P6K5 is 0:00:04.874355
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/fall
INFO:with

...

skipping /content/boundingBox/val/adl/S40P15K6
skipping /content/boundingBox/val/adl/S40P15K6
skipping /content/boundingBox/val/adl/S40P15K6


INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.347511
INFO:withlevel:Total time taken for S40P15K6 is 0:00:02.380027
INFO:withoutlevel:creating trajectory image at /content/trajectories/val/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/val/adl/S48P46K7 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.342424
INFO:withlevel:Total time taken for S48P46K7 is 0:00:02.375280
INFO:withoutlevel:creating trajectory image at /content/trajectories/val/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/val/adl/S26P28K7 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.379059
INFO:withlevel:Total time taken for S26P28K7 is 0:00:02.411792
INFO:withoutlevel:creating trajectory image at /content/trajectories/val/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/val/adl/S13P42K7 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.328180
INFO:withlevel:Total time ta

skipping /content/boundingBox/val/adl/S10P12K5
skipping /content/boundingBox/val/adl/S10P12K5
skipping /content/boundingBox/val/adl/S10P12K5
skipping /content/boundingBox/val/adl/S10P12K5


INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.395277
INFO:withlevel:Total time taken for S10P12K5 is 0:00:02.427392
INFO:withoutlevel:creating trajectory image at /content/trajectories/val/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/val/adl/S55P18K6 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.376897
INFO:withlevel:Total time taken for S55P18K6 is 0:00:02.418369
INFO:withoutlevel:creating trajectory image at /content/trajectories/test/fall
INFO:withoutlevel:Working on creating ... /content/boundingBox/test/fall/S48P46K4 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.355953
INFO:withlevel:Total time taken for S48P46K4 is 0:00:02.392869
INFO:withoutlevel:creating trajectory image at /content/trajectories/test/fall
INFO:withoutlevel:Working on creating ... /content/boundingBox/test/fall/S52P49K3 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.334745
INFO:withlevel:Total

skipping /content/boundingBox/test/adl/S44P5K6


In [ ]:
logger_t.info(f'{"="*100} \n\nSTEP 5: Create Data generators\n')
datagen = tf.keras.preprocessing.image.ImageDataGenerator()
logger.info(f'batch size: {cfg.batch_size}')
root = os.getcwd()
train = datagen.flow_from_directory(root + '/trajectories/train',target_size=(224, 224),color_mode='grayscale',batch_size=cfg.batch_size)
valid = datagen.flow_from_directory(root + '/trajectories/val',target_size=(224, 224),color_mode='grayscale',batch_size=cfg.batch_size)
test = datagen.flow_from_directory(root + '/trajectories/test',target_size=(224, 224),color_mode='grayscale',batch_size=cfg.batch_size)

INFO:withoutlevel:==================================================================================================== 

STEP 5: Create Data generators

INFO:withlevel:batch size: 32


Found 116 images belonging to 2 classes.
Found 33 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


## training pipeline


In [ ]:
from tensorflow.keras.layers import Conv2D,MaxPool2D,GlobalAveragePooling2D,Dense,Dropout
from tensorflow.keras.metrics import Precision, Recall
logger_t.info(f'{"="*100} \n\nSTEP 5: Create Model and train the model\n')



model = tf.keras.Sequential([
              tf.keras.Input(shape=(224,224,1)),
              Conv2D(32,3,strides=2,padding='same',activation='relu',use_bias=False),
              MaxPool2D(),
              Conv2D(64,3,strides=2,padding='same',activation='relu',use_bias=False),
              MaxPool2D(),
              Conv2D(128,3,strides=2,padding='same',activation='relu',use_bias=False),
              MaxPool2D(),
              GlobalAveragePooling2D(),
              Dense(128),
              Dropout(0.5),
              Dense(64),
              Dropout(0.5),
              Dense(2,activation='sigmoid')])


model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', Precision(), Recall()])
logger.info(f'{model.summary()}')



INFO:withoutlevel:==================================================================================================== 

STEP 5: Create Model and train the model



INFO:withlevel:None


Model: "sequential_335"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 112, 112, 32)      288       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 64)        18432     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 128)         73728     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 3, 3, 128)         0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 128)            

In [ ]:
best_model = wandb.restore('model-best.h5', run_path='manikandan/fall_detection/2u91085i')
best_model.name

'/content/wandb/run-20210727_165413-385vc4qg/files/model-best.h5'

In [ ]:
best_model = tf.keras.models.load_model(best_model.name)
best_model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-2), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', Precision(), Recall()])

In [ ]:
best_model.evaluate(test)

1/1 [==============================] - 1s 648ms/step - loss: 0.6478 - accuracy: 0.6667 - precision_3: 0.5714 - recall_3: 0.8889


[0.6477677822113037,
 0.6666666865348816,
 0.5714285969734192,
 0.8888888955116272]

In [ ]:
print('test')
best_model.evaluate(test)
print('validation')
best_model.evaluate(valid)
print('train')
best_model.evaluate(train)


test
1/1 [==============================] - 0s 105ms/step - loss: 0.6478 - accuracy: 0.6667 - precision_3: 0.5714 - recall_3: 0.8889
validation
2/2 [==============================] - 0s 15ms/step - loss: 0.6850 - accuracy: 0.4242 - precision_3: 0.5714 - recall_3: 0.8485
train
4/4 [==============================] - 0s 95ms/step - loss: 0.6103 - accuracy: 0.8190 - precision_3: 0.6855 - recall_3: 0.9397


[0.6103118658065796, 0.818965494632721, 0.6855345964431763, 0.9396551847457886]

In [ ]:
from wandb.keras import WandbCallback
history = best_model.fit_generator(generator=train,
                              validation_data=valid,
                              epochs=cfg.epochs,
                              verbose=1,
                              callbacks=[WandbCallback()])


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
4/4 [==============================] - 2s 358ms/step - loss: 0.6359 - accuracy: 0.6466 - precision_3: 0.6290 - recall_3: 0.6724 - val_loss: 0.7290 - val_accuracy: 0.6061 - val_precision_3: 0.6129 - val_recall_3: 0.5758
Epoch 2/15
4/4 [==============================] - 1s 331ms/step - loss: 0.4022 - accuracy: 0.8362 - precision_3: 0.8348 - recall_3: 0.8276 - val_loss: 1.2015 - val_accuracy: 0.5152 - val_precision_3: 0.5152 - val_recall_3: 0.5152
Epoch 3/15
4/4 [==============================] - 1s 374ms/step - loss: 0.1368 - accuracy: 0.9483 - precision_3: 0.9478 - recall_3: 0.9397 - val_loss: 1.8615 - val_accuracy: 0.5758 - val_precision_3: 0.5758 - val_recall_3: 0.5758
Epoch 4/15
4/4 [==============================] - 1s 326ms/step - loss: 0.0454 - accuracy: 0.9828 - precision_3: 0.9828 - recall_3: 0.9828 - val_loss: 3.3253 - val_accuracy: 0.6364 - val_precision_3: 0.6471 - val_recall_3: 0.6667
Epoch 5/15
4/4 [==============================] - 1s 363ms/step - loss: 0.0089 -

In [ ]:
results = best_model.evaluate(test)
wandb.summary['test_loss'] = results[0]
wandb.summary['test_accuracy'] = results[1]
wandb.summary['test_precision'] = results[2]
wandb.summary['test_recall'] = results[3]

1/1 [==============================] - 0s 98ms/step - loss: 2.0026 - accuracy: 0.5000 - precision_3: 0.4737 - recall_3: 0.5000


## Artifact

In [ ]:
bbox = wandb.Artifact('boundingBox', type='dataset')
bbox.add_dir('boundingBox')
wandb.log_artifact(bbox)

wandb: Adding directory to artifact (./boundingBox)... Done. 20.7s


In [ ]:
trajectory = wandb.Artifact('trajectory', type='dataset')
trajectory.add_dir('trajectories')
wandb.log_artifact(trajectory)

wandb: Adding directory to artifact (./trajectories)... Done. 0.5s


In [ ]:
 wandb.finish()